In [4]:
import pandas as pd
import numpy as np
import re
import string
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from textblob import TextBlob

# Load structured data
df = pd.read_csv(r"C:\Users\Admin\Documents\slash mark\task2(2)_dataset.csv")

# Placeholder: Load customer reviews (replace with actual dataset or scraping method)
reviews = pd.DataFrame({
    "Product_ID": ["P00069042", "P00248942", "P00087842", "P00085442"],
    "Review_Text": [
        "Great product! Loved the Black Friday discount.",
        "Not satisfied with the quality. Delivery was late.",
        "Amazing deal. Will buy again next year!",
        "Price was high even on sale. Not worth it."
    ]
})

# Text preprocessing function
def clean_text(text):
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)
    text = re.sub(r"\d+", "", text)  # Remove numbers
    return text

reviews["Cleaned_Review"] = reviews["Review_Text"].apply(clean_text)

# Sentiment Analysis
reviews["Sentiment"] = reviews["Cleaned_Review"].apply(lambda x: TextBlob(x).sentiment.polarity)

# Merge sentiment with structured data
df = df.merge(reviews, on="Product_ID", how="left")
df["Sentiment"] = df["Sentiment"].fillna(0)


# Encode categorical columns
from sklearn.preprocessing import LabelEncoder

# Identify categorical columns
categorical_cols = ["Gender", "Age", "City_Category", "Stay_In_Current_City_Years"]

# Apply Label Encoding
encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col])

# Feature selection
X = df.drop(columns=["User_ID", "Product_ID", "Purchase", "Review_Text", "Cleaned_Review"])
y = df["Purchase"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
model = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42)
model.fit(X_train, y_train)

# Predict & evaluate
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")


RMSE: 2957.6020458439425
